<a href="https://colab.research.google.com/github/hojuna/Black-box_Optimization_dacon/blob/main/Reduce_Memory_Usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Reduce Memory Usage

데이터를 경량화 시키고 저번에 메모리 부족으로 돌리지 못한 모델 돌려 보자

돌려보니 뭐 다른게 없다. ㅋ 모르겠네 오히려 과대적합 증상을 보임

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Reduce Memory Usage
# reference : https://www.kaggle.com/code/arjanso/reducing-dataframe-memory-size-by-65 @ARJANGROEN

def reduce_memory_usage(df):

    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype.name
        if ((col_type != 'datetime64[ns]') & (col_type != 'category')):
            if (col_type != 'object'):
                c_min = df[col].min()
                c_max = df[col].max()

                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)

                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        pass
            else:
                df[col] = df[col].astype('category')
    mem_usg = df.memory_usage().sum() / 1024**2
    print("Memory usage became: ",mem_usg," MB")

    return df

In [ ]:
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.model_selection import RandomizedSearchCV


# Load the data
train_df_org = pd.read_csv('/content/drive/MyDrive/dacon/2024_8_data/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/dacon/2024_8_data/test.csv')

In [ ]:
train_df_org_reduced = reduce_memory_usage(train_df_org)
test_df_reduced = reduce_memory_usage(test_df)

Memory usage of dataframe is 3.98 MB
Memory usage became:  2.38531494140625  MB
Memory usage of dataframe is 0.46 MB
Memory usage became:  0.27829742431640625  MB


In [ ]:
# df_shuffled= train_df_org
df_shuffled= train_df_org_reduced

# 균등하게 나누기 위해 임의로 데이터를 8:2로 분할
split_index = int(len(df_shuffled) * 0.8)
train_df, valiation_df = df_shuffled[:split_index], df_shuffled[split_index:]


In [ ]:

def val_score(model, valiation_df):
  id_y=pd.DataFrame()

  top_5_percent_valiation_df = valiation_df.sort_values(by='y').reset_index(drop=True)[-int(0.05 * len(valiation_df)):]
  #id만 분리 valiation_df, y값 제거
  X = valiation_df.iloc[:, 1:-1]
  id_y['ID']=valiation_df.iloc[:,0]

  #valiation_df y값 예측
  y_pred=model.predict(X)
  id_y['y_pred']=y_pred

  # print(id_x.head)
  # #id 복원

  #상위 10% valiation_df 구하기
  top_10_percent_pred_valiation=id_y.sort_values(by='y_pred').reset_index(drop=True)[-int(0.10 * len(id_y)):]

  # ID 값에서 공백 제거
  top_5_percent_valiation_df['ID'] = top_5_percent_valiation_df['ID'].str.strip()
  top_10_percent_pred_valiation['ID'] = top_10_percent_pred_valiation['ID'].str.strip()

  #top_5_percent_valiation_df 중  상위 10%에 포함되는 비율을 구해서 반환
  return top_5_percent_valiation_df['ID'].isin(top_10_percent_pred_valiation['ID']).sum()/len(top_5_percent_valiation_df)

In [ ]:

X = train_df.iloc[:, 1:-1]  # Features
y = train_df['y']  # Target

In [ ]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
import numpy as np
from sklearn.utils import shuffle

# 데이터 샘플링
X_sample, y_sample = shuffle(X, y, random_state=42)  # 데이터 샘플링
X_sample = X_sample[:10000]  #10000개의 샘플 사용
y_sample = y_sample[:10000]

# KernelRidge 모델 초기화
model = KernelRidge()

# 하이퍼파라미터 범위 설정 (더 좁은 범위)
# param_distributions = {
#     'alpha': uniform(1e-2, 1e1),  # Regularization strength (범위 축소)
#     'kernel': ['linear', 'rbf'],  # Kernel types (범위 축소)
#     'gamma': uniform(1e-2, 1e-1),  # Kernel coefficient (범위 축소)
#     'degree': [2, 3],  # Degree of the polynomial kernel (적당한 값)
#     'coef0': uniform(0, 1)  # Independent term (범위 축소)
# }
param_distributions = {
    'alpha': uniform(1e-4, 1e2),  # Regularization strength: 범위를 넓혀 작은 값부터 큰 값까지 탐색
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],  # Kernel types: 다양한 커널 옵션 추가
    'gamma': uniform(1e-4, 1e1),  # Kernel coefficient: 범위를 넓혀 작은 값부터 큰 값까지 탐색
    'degree': [2,3,4,5, 6],  # Degree of the polynomial kernel: 다양한 다항식 차수 탐색
    'coef0': uniform(-1, 2)  # Independent term: 음수 값도 포함하여 범위 확장
}


# RandomizedSearchCV 설정
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_distributions,
    n_iter=30,  # 시도할 하이퍼파라미터 조합의 수
    cv=3,  # 3-fold cross-validation
    scoring='neg_mean_squared_error',  # 평가 척도
    random_state=42,  # 결과 재현을 위한 랜덤 시드
    n_jobs=-1,  # 병렬처리
    verbose=2  # 진행 상황 출력
)

# 모델 훈련 및 하이퍼파라미터 튜닝
random_search.fit(X_sample, y_sample)

# 최적의 모델 및 하이퍼파라미터 확인
best_model = random_search.best_estimator_
best_params = random_search.best_params_
print("Best KernelRidge model:", best_model)
print("Best parameters:", best_params)

# 성능 평가
score=val_score(best_model,valiation_df)
print("KernelRidge (with tuned hyperparameters):", score)

#KernelRidge (with tuned hyperparameters): 0.8778054862842892
# KernelRidge (with tuned hyperparameters): 0.8802992518703242 경향화 후 10000개
# KernelRidge (with tuned hyperparameters): 0.875~~~~          경량화 후 20000개 오히려 더 떨어짐;;
# 30000개는 터짐

Fitting 3 folds for each of 30 candidates, totalling 90 fits
Best KernelRidge model: KernelRidge(alpha=84.89148242660839, coef0=0.4434590423297464, degree=4,
            gamma=5.227428293819941, kernel='poly')
Best parameters: {'alpha': 84.89148242660839, 'coef0': 0.4434590423297464, 'degree': 4, 'gamma': 5.227428293819941, 'kernel': 'poly'}
KernelRidge (with tuned hyperparameters): 0.8802992518703242


In [ ]:
model = KernelRidge(
    alpha=0.06522117123602399,
    coef0=0.8154614284548342,
    degree=2,
    gamma=0.081134195274865,
    kernel='linear'
)

# 모델 학습
model.fit(X, y)


# 성능 평가
score=val_score(model,valiation_df)
print("KernelRidge (with tuned hyperparameters):", score)

KernelRidge (with tuned hyperparameters): 0.8778054862842892


In [ ]:
print(best_params)

{'alpha': 0.3242918568673425, 'coef0': 0.6364104112637804, 'degree': 3, 'gamma': 0.06632755719763837, 'kernel': 'rbf'}


In [ ]:


# Predict on test data
X_test = test_df.iloc[:, 1:]
y_pred = best_model.predict(X_test)

# Identify top 10% of predicted values
threshold = np.percentile(y_pred, 90)
top_10_percent_mask = y_pred >= threshold

# Create submission file
submission_df = pd.read_csv('/content/drive/MyDrive/dacon/2024_8_data/submission/sample_submission.csv')
submission_df['y'] = y_pred
submission_df.to_csv('/content/drive/MyDrive/dacon/2024_8_data/submission/Reduce3_KernelRidge_submission.csv', index=False)